- What are the most reviewed Amazon products?
- What are the initial and current number of customer reviews for each product?
- How do the reviews in the first 90 days after a product launch compare to the price of the product?
- How do the reviews in the first 90 days after a product launch compare to the days available for sale?
- Map the keywords in the review text against the review ratings to help train sentiment models.

In [2]:
# https://www.kaggle.com/general/74235
! pip install -q kaggle
from google.colab import files
files.upload()
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets download -d datafiniti/consumer-reviews-of-amazon-products

Saving kaggle.json to kaggle.json
 55% 9.00M/16.3M [00:01<00:01, 5.31MB/s]
100% 16.3M/16.3M [00:01<00:00, 10.5MB/s]


In [3]:
!unzip consumer-reviews-of-amazon-products.zip

Archive:  consumer-reviews-of-amazon-products.zip
  inflating: 1429_1.csv              
  inflating: Datafiniti_Amazon_Consumer_Reviews_of_Amazon_Products.csv  
  inflating: Datafiniti_Amazon_Consumer_Reviews_of_Amazon_Products_May19.csv  


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import re
import pickle
from nltk.corpus import stopwords
import nltk


%matplotlib inline

In [3]:
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [5]:
reviews = pd.read_csv("./1429_1.csv")

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (1,10) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [6]:
reviews.head()

,id,name,asins,brand,categories,keys,manufacturer,reviews.date,reviews.dateAdded,reviews.dateSeen,reviews.didPurchase,reviews.doRecommend,reviews.id,reviews.numHelpful,reviews.rating,reviews.sourceURLs,reviews.text,reviews.title,reviews.userCity,reviews.userProvince,reviews.username
0,AVqkIhwDv8e3D1O-lebb,"All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi,...",B01AHB9CN2,Amazon,"Electronics,iPad & Tablets,All Tablets,Fire Ta...","841667104676,amazon/53004484,amazon/b01ahb9cn2...",Amazon,2017-01-13T00:00:00.000Z,2017-07-03T23:33:15Z,"2017-06-07T09:04:00.000Z,2017-04-30T00:45:00.000Z",NaN,True,NaN,0.0,5.0,http://reviews.bestbuy.com/3545/5620406/review...,This product so far has not disappointed. My c...,Kindle,NaN,NaN,Adapter
1,AVqkIhwDv8e3D1O-lebb,"All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi,...",B01AHB9CN2,Amazon,"Electronics,iPad & Tablets,All Tablets,Fire Ta...","841667104676,amazon/53004484,amazon/b01ahb9cn2...",Amazon,2017-01-13T00:00:00.000Z,2017-07-03T23:33:15Z,"2017-06-07T09:04:00.000Z,2017-04-30T00:45:00.000Z",NaN,True,NaN,0.0,5.0,http://reviews.bestbuy.com/3545/5620406/review...,great for beginner or experienced person. Boug...,very fast,NaN,NaN,truman
2,AVqkIhwDv8e3D1O-lebb,"All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi,...",B01AHB9CN2,Amazon,"Electronics,iPad & Tablets,All Tablets,Fire Ta...","841667104676,amazon/53004484,amazon/b01ahb9cn2...",Amazon,2017-01-13T00:00:00.000Z,2017-07-03T23:33:15Z,"2017-06-07T09:04:00.000Z,2017-04-30T00:45:00.000Z",NaN,True,NaN,0.0,5.0,http://reviews.bestbuy.com/3545/5620406/review...,Inexpensive tablet for him to use and learn on...,Beginner tablet for our 9 year old son.,NaN,NaN,DaveZ
3,AVqkIhwDv8e3D1O-lebb,"All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi,...",B01AHB9CN2,Amazon,"Electronics,iPad & Tablets,All Tablets,Fire Ta...","841667104676,amazon/53004484,amazon/b01ahb9cn2...",Amazon,2017-01-13T00:00:00.000Z,2017-07-03T23:33:15Z,"2017-06-07T09:04:00.000Z,2017-04-30T00:45:00.000Z",NaN,True,NaN,0.0,4.0,http://reviews.bestbuy.com/3545/5620406/review...,I've had my Fire HD 8 two weeks now and I love...,Good!!!,NaN,NaN,Shacks
4,AVqkIhwDv8e3D1O-lebb,"All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi,...",B01AHB9CN2,Amazon,"Electronics,iPad & Tablets,All Tablets,Fire Ta...","841667104676,amazon/53004484,amazon/b01ahb9cn2...",Amazon,2017-01-12T00:00:00.000Z,2017-07-03T23:33:15Z,"2017-06-07T09:04:00.000Z,2017-04-30T00:45:00.000Z",NaN,True,NaN,0.0,5.0,http://reviews.bestbuy.com/3545/5620406/review...,I bought this for my grand daughter when she c...,Fantastic Tablet for kids,NaN,NaN,explore42


In [7]:
reviews.columns

Index(['id', 'name', 'asins', 'brand', 'categories', 'keys', 'manufacturer',
       'reviews.date', 'reviews.dateAdded', 'reviews.dateSeen',
       'reviews.didPurchase', 'reviews.doRecommend', 'reviews.id',
       'reviews.numHelpful', 'reviews.rating', 'reviews.sourceURLs',
       'reviews.text', 'reviews.title', 'reviews.userCity',
       'reviews.userProvince', 'reviews.username'],
      dtype='object')

In [8]:
len(reviews)

34660

In [9]:
reviews = reviews[['reviews.text', 'reviews.title']]
reviews.columns = ['text', 'title']

In [11]:
reviews.head()

,text,title
0,This product so far has not disappointed. My c...,Kindle
1,great for beginner or experienced person. Boug...,very fast
2,Inexpensive tablet for him to use and learn on...,Beginner tablet for our 9 year old son.
3,I've had my Fire HD 8 two weeks now and I love...,Good!!!
4,I bought this for my grand daughter when she c...,Fantastic Tablet for kids


In [12]:
reviews.dropna(subset=['text', 'title'], inplace=True)

In [13]:
reviews.head(100)

,text,title
0,This product so far has not disappointed. My c...,Kindle
1,great for beginner or experienced person. Boug...,very fast
2,Inexpensive tablet for him to use and learn on...,Beginner tablet for our 9 year old son.
3,I've had my Fire HD 8 two weeks now and I love...,Good!!!
4,I bought this for my grand daughter when she c...,Fantastic Tablet for kids
...,...,...
95,It's a mini computer packed in a handheld devi...,Think it is a good product
96,Nice for a smaller tablet. It's good for Inter...,User friendly
97,Purchased as a door prize gift given with a on...,Excellent -- purchased for door prize
98,I have 4 other kinds of tables. This the best ...,Very versatile tablet.


downoading google word2vec embeddings
### https://www.quora.com/How-can-I-download-the-Google-news-word2vec-pretrained-model-from-a-Ubuntu-terminal



In [86]:
!wget https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz

--2020-11-18 15:00:54--  https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.217.46.166
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.217.46.166|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1647046227 (1.5G) [application/x-gzip]
Saving to: ‘GoogleNews-vectors-negative300.bin.gz.1’

GoogleNews-vectors- 100%[===================>]   1.53G  35.0MB/s    in 46s     

2020-11-18 15:01:40 (34.3 MB/s) - ‘GoogleNews-vectors-negative300.bin.gz.1’ saved [1647046227/1647046227]



In [89]:
!gunzip GoogleNews-vectors-negative300.bin.gz

In [14]:
from gensim.models import KeyedVectors

embeddings = KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin', binary=True)

In [15]:
all_docs = reviews.text.to_list()

In [16]:
def remove_stop_words(text):
    return ' '.join([word for word in text.split() if word not in stop_words])

def process_text(text):
    text = re.sub(r"[^A-Za-z0-9\`]", " ", text)
    text = re.sub(" \d+", " ", text)
    text = re.sub(r"\'s", " 's ", text)
    text = re.sub(r"n\'t", " 't ", text)
    text = re.sub(r"\'re", " 're ", text)
    text = re.sub(r"\'d", " 'd ", text)
    text = re.sub(r"\'ll", " 'll ", text)
    text = re.sub(r",", " ", text)
    text = re.sub(r"\.", " ", text)
    text = re.sub(r"!", " ", text)
    text = re.sub(r"\(", " ( ", text)
    text = re.sub(r"\)", " ) ", text)
    text = re.sub(r"\?", " ", text)
    text = re.sub(r"\s{2,}", " ", text)
    text = remove_stop_words(text)
    words = text.split(' ') 
    return words

In [17]:
def get_document_embedding(tweet, embeddings): 
    doc_embedding = np.zeros(300)

    processed_doc = process_text(tweet)
    for word in processed_doc:
        if word in embeddings:
          doc_embedding += embeddings[word]
    return doc_embedding

In [18]:
def get_document_vecs(all_docs, embeddings):


    ind2Doc_dict = {}

    document_vec_l = []

    for i, doc in enumerate(all_docs):

        doc_embedding = get_document_embedding(doc, embeddings)

        ind2Doc_dict[i] = doc_embedding

        document_vec_l.append(doc_embedding)


    document_vec_matrix = np.vstack(document_vec_l)

    return document_vec_matrix, ind2Doc_dict

In [19]:
document_vecs, ind2Review = get_document_vecs(reviews.text.to_list(), embeddings)

In [20]:
print(f"length of dictionary {len(ind2Review)}")
print(f"shape of document_vecs {document_vecs.shape}")

length of dictionary 34654
shape of document_vecs (34654, 300)


In [24]:
my_tweet = 'tastes good in the first time'
# my_tweet = process_text(my_tweet)
tweet_embedding = get_document_embedding(my_tweet, embeddings)

In [22]:
def cosine_similarity(A, B):
    
    dot = np.sum(np.dot(A, B))
    norma = np.linalg.norm(A)
    normb = np.linalg.norm(B)
    cos = dot / (norma * normb)

    return cos

In [25]:
idx = np.argmax(cosine_similarity(document_vecs, tweet_embedding))
print(reviews.text.to_list()[idx])

This product so far has not disappointed. My children love to use it and I like the ability to monitor control what content they see with ease.


In [26]:
N_VECS = len(all_docs)       # This many vectors.
N_DIMS = len(ind2Review[1])     # Vector dimensionality.
print(f"Number of vectors is {N_VECS} and each has {N_DIMS} dimensions.")

Number of vectors is 34654 and each has 300 dimensions.


In [27]:
# The number of planes. We use log2(625) to have ~16 vectors/bucket.
N_PLANES = 10
# Number of times to repeat the hashing to improve the search.
N_UNIVERSES = 25

In [28]:
np.random.seed(0)
planes_l = [np.random.normal(size=(N_DIMS, N_PLANES))
            for _ in range(N_UNIVERSES)]

In [29]:
def hash_value_of_vector(v, planes):

    dot_product = np.dot(v, planes)

    sign_of_dot_product = np.sign(dot_product)
    sign_of_dot_product = np.squeeze(sign_of_dot_product)
    h = [0 if i < 0 else 1 for i in sign_of_dot_product]
    h = np.squeeze(h)
    hash_value = 0

    n_planes = planes.shape[1]
    for i in range(n_planes):
        hash_value += 2 ** i * h[i]
    hash_value = int(hash_value)

    return hash_value

In [30]:
np.random.seed(0)
idx = 0
planes = planes_l[idx]  # get one 'universe' of planes to test the function
vec = np.random.rand(1, 300)
print(f" The hash value for this vector,",
      f"and the set of planes at index {idx},",
      f"is {hash_value_of_vector(vec, planes)}")

 The hash value for this vector, and the set of planes at index 0, is 768


In [31]:
def make_hash_table(vecs, planes):
  
    num_of_planes = planes.shape[1]
    num_buckets = 2 ** num_of_planes
    hash_table = {k: [] for k in range(num_buckets)}

    id_table = {k: [] for k in range(num_buckets)}

    for i, v in enumerate(vecs):
        h = hash_value_of_vector(v, planes)

        hash_table[h].append(v)

        id_table[h].append(i)

    return hash_table, id_table

In [32]:
np.random.seed(0)
planes = planes_l[0]  # get one 'universe' of planes to test the function
vec = np.random.rand(1, 300)
tmp_hash_table, tmp_id_table = make_hash_table(document_vecs, planes)

print(f"The hash table at key 0 has {len(tmp_hash_table[0])} document vectors")
print(f"The id table at key 0 has {len(tmp_id_table[0])}")
print(f"The first 5 document indices stored at key 0 of are {tmp_id_table[0][0:5]}")

The hash table at key 0 has 1 document vectors
The id table at key 0 has 1
The first 5 document indices stored at key 0 of are [34314]


In [33]:
hash_tables = []
id_tables = []
for universe_id in range(N_UNIVERSES):  # there are 25 hashes
    print('working on hash universe #:', universe_id)
    planes = planes_l[universe_id]
    hash_table, id_table = make_hash_table(document_vecs, planes)
    hash_tables.append(hash_table)
    id_tables.append(id_table)

working on hash universe #: 0
working on hash universe #: 1
working on hash universe #: 2
working on hash universe #: 3
working on hash universe #: 4
working on hash universe #: 5
working on hash universe #: 6
working on hash universe #: 7
working on hash universe #: 8
working on hash universe #: 9
working on hash universe #: 10
working on hash universe #: 11
working on hash universe #: 12
working on hash universe #: 13
working on hash universe #: 14
working on hash universe #: 15
working on hash universe #: 16
working on hash universe #: 17
working on hash universe #: 18
working on hash universe #: 19
working on hash universe #: 20
working on hash universe #: 21
working on hash universe #: 22
working on hash universe #: 23
working on hash universe #: 24


In [35]:
def nearest_neighbor(v, candidates, k=1):
    """
    Input:
      - v, the vector you are going find the nearest neighbor for
      - candidates: a set of vectors where we will find the neighbors
      - k: top k nearest neighbors to find
    Output:
      - k_idx: the indices of the top k closest vectors in sorted form
    """
    ### START CODE HERE (REPLACE INSTANCES OF 'None' with your code) ###
    similarity_l = []

    # for each candidate vector...
    for row in candidates:
        # get the cosine similarity
        cos_similarity = cosine_similarity(v, row)

        # append the similarity to the list
        similarity_l.append(cos_similarity)
        
    # sort the similarity list and get the indices of the sorted list
    sorted_ids = np.argsort(similarity_l)

    # get the indices of the k most similar candidate vectors
    k_idx = sorted_ids[-k:]
    ### END CODE HERE ###
    return k_idx

In [36]:
def approximate_knn(doc_id, v, planes_l, k=1, num_universes_to_use=N_UNIVERSES):
    assert num_universes_to_use <= N_UNIVERSES

    vecs_to_consider_l = list()

    ids_to_consider_l = list()

    ids_to_consider_set = set()

    for universe_id in range(num_universes_to_use):

        planes = planes_l[universe_id]

        hash_value = hash_value_of_vector(v, planes)

        hash_table = hash_tables[universe_id]

        document_vectors_l = hash_table[hash_value]

        id_table = id_tables[universe_id]

        new_ids_to_consider = id_table[hash_value]
        if doc_id in new_ids_to_consider:
            new_ids_to_consider.remove(doc_id)
            print(f"removed doc_id {doc_id} of input vector from new_ids_to_search")

        # loop through the subset of document vectors to consider
        for i, new_id in enumerate(new_ids_to_consider):

            if new_id not in ids_to_consider_set:
                document_vector_at_i = document_vectors_l[i]

                vecs_to_consider_l.append(document_vector_at_i)
                ids_to_consider_l.append(new_id)

                ids_to_consider_set.add(new_id)

    print("Fast considering %d vecs" % len(vecs_to_consider_l))

    vecs_to_consider_arr = np.array(vecs_to_consider_l)

    nearest_neighbor_idx_l = nearest_neighbor(v, vecs_to_consider_arr, k=k)

    nearest_neighbor_ids = [ids_to_consider_l[idx]
                            for idx in nearest_neighbor_idx_l]

    return nearest_neighbor_ids

In [45]:
doc_id = 34000
doc_to_search = all_docs[doc_id]
vec_to_search = document_vecs[doc_id]

In [48]:
nearest_neighbor_ids = approximate_knn(
    doc_id, vec_to_search, planes_l, k=3, num_universes_to_use=5)

Fast considering 12205 vecs


In [49]:
print(f"Nearest neighbors for document {doc_id}")
print(f"Document contents: {doc_to_search}")
print("")

for neighbor_id in nearest_neighbor_ids:
    print(f"Nearest neighbor at document id {neighbor_id}")
    print(f"document contents: {all_docs[neighbor_id]}")

Nearest neighbors for document 34000
Document contents: Fast processor to run my kodi. I love it. I only use it for that.

Nearest neighbor at document id 34165
document contents: I love this divice, it has an amazing image, I will recomend it to any body, and ypu can install a lot of good apps
Nearest neighbor at document id 34026
document contents: Bought the Fire because I wanted to get Vue from Playstation which will not stream on Roku. First of all I do not like the Fire's layout. It is very cluttered and harder to find what you are looking for than on Roku. Also I had trouble with the streaming and extreme amounts of buffering while on WIFI. My modem was only a few feet away from the Fire. Roku almost never had buffering/streaming issues. I had to plug the Fire directly into the modem in order for it to work. This did take care of the buffering problems. I gave it a good for working when plugged into the modem. For WIFI only it would get a rating of POOR. I am very glad I did not